<a href="https://colab.research.google.com/github/gowen100/Jane-Street-Solutions/blob/master/JaneSt_Mar19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
from skimage.morphology import label
import requests
from bs4 import BeautifulSoup
import copy

# Using old puzzles to inform my solution for the Dec 20 puzzle
# Could do far more on limiting the permissable placements. 
# Especially including the looks in limiting what could be placed 
# in a row which would speed up the bottom left puzzle which at present is very slow

### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2019/03/20190306_20_4_7.png" width="500" height="600">

In [2]:
# Set up the grids to be solved

grid4 =np.array([[0,0,0,7,0,0,0],
                 [5,0,0,0,0,6,0],
                 [0,0,2,0,0,0,0],
                 [5,0,0,0,0,0,7],
                 [0,0,0,0,7,0,0],
                 [0,7,0,0,0,0,3],
                 [0,0,0,6,0,0,0]])

top_looks4 = [5,0,5,0,2,0,7]
bot_looks4 = [4,0,6,0,4,0,3]
left_looks4 = [5,0,0,0,4,0,0]
right_looks4 = [0,0,7,0,0,0,4]

grid3 =np.array([[0,0,4,0,0,0,0],
                 [0,4,0,0,0,0,0],
                 [4,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,7],
                 [0,0,0,0,0,5,0],
                 [0,0,0,0,7,0,0]])

top_looks3 = [0,0,0,6,3,7,0]
bot_looks3 = [0,3,6,7,0,0,0]
left_looks3 = [0,0,0,2,6,3,0]
right_looks3 = [0,4,3,5,0,0,0]

grid2 =np.array([[0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0]])

top_looks2 = [6,4,4,6,7,3,3]
bot_looks2 = [2,6,7,7,2,4,7]
left_looks2 = [4,6,5,7,5,2,7]
right_looks2 = [3,6,3,3,7,6,7]

grid1=np.array([[0,7,6,0,0,0,0],
                [0,0,0,6,6,0,0],
                [5,0,0,0,0,0,0],
                [0,6,0,0,0,4,0],
                [0,0,0,0,0,0,6],
                [0,0,4,7,0,0,0],
                [0,0,0,0,7,7,5]])

top_looks1 = [0,0,0,0,0,5,7]
bot_looks1 = [7,2,0,0,0,0,0]
left_looks1 = [0,3,0,0,5,0,1]
right_looks1 = [7,2,0,0,0,0,0]


# Use -1 for empty rather than zero to track completion of cells
grid1[grid1==0]=-1
grid2[grid2==0]=-1
grid3[grid3==0]=-1
grid4[grid4==0]=-1

In [3]:
# Set up a class to represent the matrix
# methods to 
# - print
# - check a possible placement
#   - to check 2x2 placement isn't violated
#   - to check there aren't too many small numbers in a row or column
#   - to check looks aren't violated
#  - Recursively solve the problem

class Matrix():
    def __init__(self,input_grid,top_looks,bot_looks,left_looks,right_looks):
        self.grid = input_grid
        self.top_looks = top_looks
        self.bot_looks = bot_looks
        self.left_looks =left_looks
        self.right_looks = right_looks
        self.counts = {i:i-np.sum(self.grid==i) for i in range(8)}
        print(self.counts)
        print('The problem - Start {}'.format(time.strftime('%X')))
        self.printGrid()
    
#Function to print the grid
    def printGrid(self):
        print(self.grid)
        
#Function to check if a digit can be placed in the given block
#Various checks in the main block with more complex checks on small numbers, 2x2 and "looks" moved to separate functions
    def possible(self,row,col,digit):

        # Check row and columns don't have too many zeros
        if digit ==0:
            if np.sum(self.grid[:,col]==0) > 2:
                #print('fail1')
                return False
            if np.sum(self.grid[row,:]==0) > 2:
                #print('fail2')
                return False
            
        # Check that final number placed completes
        if digit !=0:
            if np.sum(self.grid[row,:]>0) > 2:
                if np.sum(self.grid[row,:])+digit+np.sum(self.grid[row,:]==-1)  != 20:
                    return False
                    
            if np.sum(self.grid[:,col]>0) > 2:
                if np.sum(self.grid[:,col])+digit+np.sum(self.grid[:,col]==-1)  != 20:
                    return False
        
        # Check row and columns don't exceed sum
        if np.sum(self.grid[:,col])+digit+np.sum(self.grid[:,col]==-1) > 20:
            return False
        
        if np.sum(self.grid[row,:])+digit+np.sum(self.grid[row,:]==-1)  > 20:
            return False
        
        #check 2x2
        if digit !=0:
            if self.twobytwo(row,col):
                return False
            
        # Check forbidden nums
            if self.more_constraints(self.grid[row,:],digit):
                return False
        
            if self.more_constraints(self.grid[:,col],digit):
                return False
            
            if self.check_looks(row,col,digit):
                return False
        
        #checkconnected (nice trick to use sk-image to do this)
        if np.max(label(self.grid!=0,connectivity=1)) >1 :
            return False
        
        # All tests pass return True
        return True
    
#check if placing a number violates the 2x2 rule
    def twobytwo(self,i,j):
        if (i > 0 and j > 0 and 
        self.grid[i-1,j-1] > 0 and 
        self.grid[i-1,j] > 0 and 
        self.grid[i,j-1] > 0):
            return True
    
        if (i > 0 and j < 6 and
        self.grid[i-1,j+1] > 0 and 
        self.grid[i-1,j] > 0 and 
        self.grid[i,j+1] > 0):
            return True
    
        if (i < 6 and j > 0 and
        self.grid[i+1,j-1] > 0 and 
        self.grid[i+1,j] > 0 and 
        self.grid[i,j-1] > 0):
            return True

    
        if (i < 6 and j < 6 and
        self.grid[i+1,j+1] > 0 and 
        self.grid[i+1,j] > 0 and 
        self.grid[i,j+1] > 0):
            return True
    
        return False
    
#check to see if there are too many small numbers in a row or column
    def more_constraints(self,array,digit):
        if np.sum(array==1) > 0:
            if digit < 5:
                return True
            return False
        
        if np.sum(array==2) > 0:
            if digit < 4 :
                return True
            return False
    
        if np.sum(array==3) > 0:
            if digit < 3 :
                return True
            return False
    
        if np.sum(array==4) > 0:
            if digit < 2 :
                return True
            return False
        
        return False
        
# Check to see if the looks are correct 
    def check_looks(self,row,col,digit):
        if self.left_looks[row] > 0:
            found = False
            for i in range(5):
                if found == False:
                    if self.grid[row,i] != 0:
                        found = True
                        if i == col:
                            if self.left_looks[row] != digit:
                                return True
    
        if self.top_looks[col] > 0:
            found = False
            for i in range(5):
                if found == False:
                    if self.grid[i,col] != 0:
                        found = True
                        if i == row:
                            if self.top_looks[col] != digit:
                                return True

        if self.right_looks[row] > 0:
            found = False
            for i in range(6,4,-1):
                if found == False:
                    if self.grid[row,i] != 0:
                        found = True
                        if i == col:
                            if self.right_looks[row] != digit:
                                return True
    
        if self.bot_looks[col] > 0:
            found = False
            for i in range(6,4,-1):
                if found == False:
                    if self.grid[i,col] != 0:
                        found = True
                        if i == row:
                            if self.bot_looks[col] != digit:
                                return True
        return False    
        
# =============================================================================================================================================================================
# The main function. Loop throught the cells trying to place a digit, if the digit is allowed carry on and recall the function recursively until that is not allowed. 
# Then backtrack and retry. Based on the simple soduku solver here https://medium.com/daily-python/solving-sudoku-puzzle-using-backtracking-in-python-daily-python-29-99a825042e
# which is where I started on this.

    def solve(self):
        
        for row in range(7):
            for col in range(7):
                if self.grid[row,col] == -1:
                    for digit in range(7,-1,-1):
                        if (digit ==0 or self.counts[digit]>0):
                            if self.possible(row,col,digit):
                                self.grid[row,col] = digit
                                self.counts[digit] -= 1
                                self.solve()
                                self.grid[row,col] = -1  #Backtrack step
                                self.counts[digit] += 1
                    return 
        
        print('\nThe solution - End {}'.format(time.strftime('%X')))
        self.solution = copy.copy(self.grid)

In [4]:
# Create a separate instance of the Matrix class for each puzzle and solve.
# x1 -> x4

x1 = Matrix(grid1,top_looks1,bot_looks1,left_looks1,right_looks1)
output1 = x1.solve()
print(x1.solution)
print('\n Finish {}'.format(time.strftime('%X')))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 2, 5: 3, 6: 1, 7: 3}
The problem - Start 09:39:12
[[-1  7  6 -1 -1 -1 -1]
 [-1 -1 -1  6  6 -1 -1]
 [ 5 -1 -1 -1 -1 -1 -1]
 [-1  6 -1 -1 -1  4 -1]
 [-1 -1 -1 -1 -1 -1  6]
 [-1 -1  4  7 -1 -1 -1]
 [-1 -1 -1 -1  7  7  5]]

The solution - End 09:39:15
[[3 7 6 0 4 0 0]
 [0 0 3 6 6 5 0]
 [5 5 0 6 0 4 0]
 [0 6 0 0 3 4 7]
 [5 2 7 0 0 0 6]
 [7 0 4 7 0 0 2]
 [0 0 0 1 7 7 5]]

 Finish 09:39:15


In [ ]:
x2 = Matrix(grid2,top_looks2,bot_looks2,left_looks2,right_looks2)
x2.solve()
print(x2.solution)
print('\n Finish {}'.format(time.strftime('%X')))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
The problem - Start 09:39:15
[[-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]]


In [ ]:
x3 = Matrix(grid3,top_looks3,bot_looks3,left_looks3,right_looks3)
x3.solve()
print(x3.solution)
print('\n Finish {}'.format(time.strftime('%X')))

In [ ]:
x4 = Matrix(grid4,top_looks4,bot_looks4,left_looks4,right_looks4)
x4.solve()
print(x4.solution)
print('\n Finish {}'.format(time.strftime('%X')))

In [ ]:
# Calculate the solution based on the grids

solved = np.sum((x1.solution+x2.solution+x3.solution+x4.solution)**2)
print("The solution is: ",solved)

In [ ]:
# Pull in the puzzle solution and grids from the solution page

url='https://www.janestreet.com/puzzles/solutions/march-2019-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
x =[text for text in soup.body.stripped_strings]

print(" ".join(x[7:9]))

### Solution is :

<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2019/04/20190329_20_4_7_solved.png" width="500" height="600">